<a href="https://colab.research.google.com/github/Narsi07/PRODIGY_GA_01/blob/main/Task-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importing libraries
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
from datasets import load_dataset, Dataset, DatasetDict
from transformers import DataCollatorForLanguageModeling
from transformers import EarlyStoppingCallback
import os
from torch.utils.data import DataLoader
from tqdm import tqdm
import math



In [ ]:
# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load GPT-2 model and tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
# Configure tokenizer
tokenizer.pad_token = tokenizer.eos_token

# Load and preprocess dataset with improved handling
def load_dataset_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = [line.strip() for line in f if line.strip()]

    # Chunk text into overlapping sequences for better context learning
    chunk_size = 512  # matches our max_length
    overlap = 100
    chunks = []
    for line in lines:
        words = line.split()
        for i in range(0, len(words), chunk_size - overlap):
            chunk = ' '.join(words[i:i + chunk_size])
            chunks.append(chunk)

    return DatasetDict({"train": Dataset.from_dict({"text": chunks})})



In [ ]:
# Improved tokenization with dynamic padding
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        max_length=512,
        padding=False  # We'll handle padding in the data collator
    )

# Load dataset
dataset_path = "/content/drive/MyDrive/Colab Notebooks/dataset.txt"
dataset = load_dataset_file(dataset_path)
tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["text"])



Map:   0%|          | 0/4 [00:00<?, ? examples/s]

In [ ]:
# Use data collator for efficient dynamic padding and language modeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # For causal language modeling
    return_tensors="pt"
)

# Enhanced training arguments
training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",
    overwrite_output_dir=True,
    num_train_epochs=5,  # Increased from 3
    per_device_train_batch_size=8 if torch.cuda.is_available() else 4,  # Increased batch size
    gradient_accumulation_steps=2,  # For effective larger batch sizes
    learning_rate=5e-5,  # Lower learning rate for fine-tuning
    weight_decay=0.01,  # Regularization
    warmup_steps=500,  # Learning rate warmup
    save_strategy="epoch",
    eval_strategy="steps" if len(tokenized_dataset) > 1 else "no",
    eval_steps=2000,
    logging_dir="./logs",
    logging_steps=200,
    fp16=torch.cuda.is_available(),
    load_best_model_at_end=True if len(tokenized_dataset) > 1 else False,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    report_to="tensorboard",
)



In [ ]:
# Initialize Trainer with additional features
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)] if len(tokenized_dataset) > 1 else None,
)

# Fine-tune the model with progress tracking
trainer.train()

# Save the fine-tuned model and tokenizer
model.save_pretrained("./gpt2-finetuned")
tokenizer.save_pretrained("./gpt2-finetuned")



`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


('./gpt2-finetuned/tokenizer_config.json',
 './gpt2-finetuned/special_tokens_map.json',
 './gpt2-finetuned/vocab.json',
 './gpt2-finetuned/merges.txt',
 './gpt2-finetuned/added_tokens.json')

In [ ]:
# Enhanced text generation function
def generate_text(prompt, max_length=100, temperature=0.7, top_k=50, top_p=0.9):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    outputs = model.generate(
        inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=max_length,
        num_return_sequences=2,
        do_sample=True,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        pad_token_id=tokenizer.eos_token_id,
        no_repeat_ngram_size=2,
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Genearting text using finetuned model
prompt = "Iam pirate king"
generated_text = generate_text(prompt, max_length=150, temperature=0.8)
print("\nGenerated Text:")
print(generated_text)


Generated Text:
Iam pirate king. And when he's gone, his brother goes to a place called "The Pirate's Landing" and he becomes the king of the world. I'm not sure if you've ever heard of that. But that's a good time to think about it.

So, what's the first thing that happens. The pirate's is going to have a hard time. He has to face a lot of bad guys. It's not like he has a life, but he'll face lots of good guys and bad ones. This is what makes him so special. That's how we make him special in a way that he doesn't know or that we don't. A lot more than he does. Because he is special
